<a href="https://colab.research.google.com/github/VoonaSriraj/Enhanced-Machine-Translation/blob/main/machinetranslation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# ============================================================================
# ENHANCED MACHINE TRANSLATION - ADD THIS CELL TO YOUR NOTEBOOK
# ============================================================================
"""
Add this cell to your existing machine_translation.ipynb notebook
This provides a drop-in enhancement to your existing models
"""

import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, Model
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from typing import List, Dict, Tuple
import time

# Check for enhanced dependencies
try:
    import sentencepiece as smp
    from sacrebleu import corpus_bleu
    ENHANCED_MODE = True
    print("✅ Enhanced mode: Advanced features available")
except ImportError:
    ENHANCED_MODE = False
    print("⚠️ Basic mode: Install 'sentencepiece sacrebleu' for full features")

class EnhancedConfig:
    """Configuration for enhanced models"""
    def __init__(self):
        # Model architecture
        self.d_model = 256  # Smaller for compatibility with existing notebook
        self.num_heads = 8
        self.num_layers = 4
        self.dff = 1024
        self.dropout_rate = 0.1

        # Training
        self.batch_size = 64
        self.epochs = 15
        self.learning_rate = 1e-4
        self.warmup_steps = 2000

        # Advanced features
        self.use_beam_search = True
        self.beam_size = 4
        self.length_penalty = 0.6
        self.label_smoothing = 0.1

# Enhanced Transformer implementation compatible with your existing notebook
class ModernMultiHeadAttention(layers.Layer):
    """Improved multi-head attention with better initialization"""

    def __init__(self, d_model, num_heads, **kwargs):
        super().__init__(**kwargs)
        self.num_heads = num_heads
        self.d_model = d_model

        assert d_model % self.num_heads == 0

        self.depth = d_model // self.num_heads

        self.wq = layers.Dense(d_model, kernel_initializer='glorot_uniform')
        self.wk = layers.Dense(d_model, kernel_initializer='glorot_uniform')
        self.wv = layers.Dense(d_model, kernel_initializer='glorot_uniform')

        self.dense = layers.Dense(d_model, kernel_initializer='glorot_uniform')

    def split_heads(self, x, batch_size):
        x = tf.reshape(x, (batch_size, -1, self.num_heads, self.depth))
        return tf.transpose(x, perm=[0, 2, 1, 3])

    def call(self, inputs):
        v, k, q = inputs
        batch_size = tf.shape(q)[0]

        q = self.wq(q)
        k = self.wk(k)
        v = self.wv(v)

        q = self.split_heads(q, batch_size)
        k = self.split_heads(k, batch_size)
        v = self.split_heads(v, batch_size)

        # Scaled dot-product attention
        matmul_qk = tf.matmul(q, k, transpose_b=True)

        dk = tf.cast(tf.shape(k)[-1], tf.float32)
        scaled_attention_logits = matmul_qk / tf.math.sqrt(dk)

        attention_weights = tf.nn.softmax(scaled_attention_logits, axis=-1)

        output = tf.matmul(attention_weights, v)

        output = tf.transpose(output, perm=[0, 2, 1, 3])
        output = tf.reshape(output, (batch_size, -1, self.d_model))

        output = self.dense(output)

        return output

class EnhancedTransformerBlock(layers.Layer):
    """Enhanced transformer block with pre-layer normalization"""

    def __init__(self, d_model, num_heads, dff, rate=0.1, **kwargs):
        super().__init__(**kwargs)

        self.mha = ModernMultiHeadAttention(d_model, num_heads)
        self.ffn = tf.keras.Sequential([
            layers.Dense(dff, activation='relu', kernel_initializer='glorot_uniform'),
            layers.Dense(d_model, kernel_initializer='glorot_uniform')
        ])

        self.layernorm1 = layers.LayerNormalization(epsilon=1e-6)
        self.layernorm2 = layers.LayerNormalization(epsilon=1e-6)

        self.dropout1 = layers.Dropout(rate)
        self.dropout2 = layers.Dropout(rate)

    def call(self, x, training=None):
        # Pre-layer normalization
        attn_input = self.layernorm1(x)
        attn_output = self.mha([attn_input, attn_input, attn_input])
        attn_output = self.dropout1(attn_output, training=training)
        out1 = x + attn_output  # Residual connection

        ffn_input = self.layernorm2(out1)
        ffn_output = self.ffn(ffn_input)
        ffn_output = self.dropout2(ffn_output, training=training)
        out2 = out1 + ffn_output  # Residual connection

        return out2

def enhanced_transformer_model(input_shape, output_sequence_length, english_vocab_size, french_vocab_size):
    """
    Enhanced Transformer model - replacement for your model_final function
    """
    config = EnhancedConfig()

    # Input layers
    encoder_inputs = layers.Input(shape=input_shape[1:], name='encoder_inputs')
    decoder_inputs = layers.Input(shape=(output_sequence_length,), name='decoder_inputs')

    # Embeddings
    encoder_embedding = layers.Embedding(
        english_vocab_size, config.d_model,
        mask_zero=True, name='encoder_embedding'
    )(encoder_inputs)

    decoder_embedding = layers.Embedding(
        french_vocab_size, config.d_model,
        mask_zero=True, name='decoder_embedding'
    )(decoder_inputs)

    # Positional encoding
    def add_positional_encoding(embedded_inputs):
        seq_len = tf.shape(embedded_inputs)[1]
        d_model = embedded_inputs.shape[-1]

        # Create positional encoding
        pos = tf.range(seq_len, dtype=tf.float32)[:, tf.newaxis]
        i = tf.range(d_model, dtype=tf.float32)[tf.newaxis, :]

        angle_rates = 1 / tf.pow(10000.0, (2 * (i // 2)) / d_model)
        angle_rads = pos * angle_rates

        # Apply sin to even indices
        sines = tf.sin(angle_rads[:, 0::2])
        # Apply cos to odd indices
        cosines = tf.cos(angle_rads[:, 1::2])

        pos_encoding = tf.concat([sines, cosines], axis=-1)
        pos_encoding = pos_encoding[tf.newaxis, ...]

        return embedded_inputs + pos_encoding

    # Add positional encoding
    encoder_outputs = add_positional_encoding(encoder_embedding)
    decoder_outputs = add_positional_encoding(decoder_embedding)

    # Encoder layers
    for i in range(config.num_layers):
        encoder_outputs = EnhancedTransformerBlock(
            config.d_model, config.num_heads, config.dff, config.dropout_rate,
            name=f'encoder_layer_{i}'
        )(encoder_outputs)

    # Decoder layers (simplified for seq2seq)
    for i in range(config.num_layers):
        decoder_outputs = EnhancedTransformerBlock(
            config.d_model, config.num_heads, config.dff, config.dropout_rate,
            name=f'decoder_layer_{i}'
        )(decoder_outputs)

    # Final dense layer
    outputs = layers.Dense(french_vocab_size, activation='softmax', name='output_layer')(decoder_outputs)

    # Create model
    model = Model([encoder_inputs, decoder_inputs], outputs, name='enhanced_transformer')

    # Enhanced optimizer with learning rate scheduling
    initial_learning_rate = config.learning_rate
    lr_schedule = tf.keras.optimizers.schedules.CosineDecay(
        initial_learning_rate, decay_steps=1000, alpha=0.1
    )

    optimizer = tf.keras.optimizers.Adam(
        learning_rate=lr_schedule,
        beta_1=0.9,
        beta_2=0.98,
        epsilon=1e-9
    )

    # Enhanced loss with label smoothing
    def label_smoothed_loss(y_true, y_pred):
        # Convert sparse labels to one-hot
        y_true_one_hot = tf.one_hot(tf.cast(y_true, tf.int32), french_vocab_size)

        # Apply label smoothing
        smoothing = config.label_smoothing
        y_true_smooth = y_true_one_hot * (1 - smoothing) + smoothing / french_vocab_size

        # Calculate loss
        loss = tf.keras.losses.categorical_crossentropy(y_true_smooth, y_pred)

        # Mask padding tokens (assuming 0 is padding)
        mask = tf.cast(tf.not_equal(y_true, 0), tf.float32)
        loss = loss * mask

        return tf.reduce_sum(loss) / tf.reduce_sum(mask)

    model.compile(
        optimizer=optimizer,
        loss=label_smoothed_loss,
        metrics=['accuracy']
    )

    return model

def enhanced_beam_search_decode(model, input_sequence, tokenizer, beam_size=4, max_length=50):
    """
    Beam search decoder for better translation quality
    """
    # Encode input
    encoder_input = tf.expand_dims(input_sequence, 0)

    # Initialize beam with start token (assuming 1 is start token)
    beams = [(0.0, [1])]  # (score, sequence)

    for step in range(max_length):
        all_candidates = []

        for score, sequence in beams:
            if len(sequence) > 0 and sequence[-1] == 2:  # End token
                all_candidates.append((score, sequence))
                continue

            # Prepare decoder input
            decoder_input = tf.constant([sequence + [0] * (max_length - len(sequence))])
            decoder_input = decoder_input[:, :max_length]

            # Get predictions
            predictions = model([encoder_input, decoder_input])

            # Get probabilities for next token
            next_token_probs = predictions[0, len(sequence)-1, :]

            # Get top beam_size tokens
            top_probs, top_indices = tf.nn.top_k(next_token_probs, k=beam_size)

            for i in range(beam_size):
                token_id = int(top_indices[i])
                token_prob = float(tf.nn.log_softmax(next_token_probs)[top_indices[i]])

                new_sequence = sequence + [token_id]
                new_score = score + token_prob

                # Length penalty
                length_penalty = ((5 + len(new_sequence)) / 6) ** 0.6
                normalized_score = new_score / length_penalty

                all_candidates.append((normalized_score, new_sequence))

        # Keep top beam_size candidates
        beams = sorted(all_candidates, key=lambda x: x[0], reverse=True)[:beam_size]

        # Stop if all beams end with end token
        if all(len(seq) > 0 and seq[-1] == 2 for _, seq in beams):
            break

    # Return best sequence
    return beams[0][1]

def evaluate_enhanced_model(model, test_data, tokenizer, beam_search=True):
    """
    Enhanced evaluation with multiple metrics
    """
    predictions = []
    references = []

    print("Generating predictions...")
    for i, (src, tgt) in enumerate(test_data):
        if i % 100 == 0:
            print(f"Progress: {i}/{len(test_data)}")

        if beam_search:
            pred_sequence = enhanced_beam_search_decode(model, src, tokenizer)
        else:
            # Simple greedy decoding
            encoder_input = tf.expand_dims(src, 0)
            decoder_input = tf.constant([[1] + [0] * 49])  # Start with start token

            pred = model([encoder_input, decoder_input])
            pred_sequence = tf.argmax(pred[0], axis=-1).numpy().tolist()

        # Convert back to text (simplified)
        pred_text = ' '.join([str(token) for token in pred_sequence if token not in [0, 1, 2]])
        ref_text = ' '.join([str(token) for token in tgt if token not in [0, 1, 2]])

        predictions.append(pred_text)
        references.append(ref_text)

    # Calculate BLEU score
    if ENHANCED_MODE:
        try:
            bleu_score = corpus_bleu(predictions, [[ref] for ref in references]).score
            print(f"📊 sacreBLEU score: {bleu_score:.2f}")
        except:
            print("⚠️ Could not calculate sacreBLEU")

    # Simple BLEU approximation
    def simple_bleu(pred, ref):
        pred_tokens = set(pred.split())
        ref_tokens = set(ref.split())
        if len(pred_tokens) == 0:
            return 0.0
        return len(pred_tokens & ref_tokens) / len(pred_tokens)

    simple_bleu_scores = [simple_bleu(p, r) for p, r in zip(predictions, references)]
    avg_simple_bleu = np.mean(simple_bleu_scores)

    print(f"📊 Simple BLEU score: {avg_simple_bleu:.4f}")

    return {
        'predictions': predictions,
        'references': references,
        'simple_bleu': avg_simple_bleu,
        'bleu_scores': simple_bleu_scores
    }

def create_enhanced_datasets(english_sentences, french_sentences, test_size=0.2, val_size=0.1):
    """
    Create proper train/validation/test splits
    """
    print("🔄 Creating enhanced data splits...")

    # First split: separate test set
    en_temp, en_test, fr_temp, fr_test = train_test_split(
        english_sentences, french_sentences,
        test_size=test_size, random_state=42
    )

    # Second split: separate validation set
    val_size_adjusted = val_size / (1 - test_size)
    en_train, en_val, fr_train, fr_val = train_test_split(
        en_temp, fr_temp,
        test_size=val_size_adjusted, random_state=42
    )

    print(f"📊 Dataset splits:")
    print(f"  - Training: {len(en_train)} pairs")
    print(f"  - Validation: {len(en_val)} pairs")
    print(f"  - Test: {len(en_test)} pairs")

    return {
        'train': (en_train, fr_train),
        'val': (en_val, fr_val),
        'test': (en_test, fr_test)
    }

def enhanced_training_pipeline(english_sentences, french_sentences):
    """
    Complete enhanced training pipeline
    """
    print("🚀 Starting Enhanced Machine Translation Pipeline")
    print("=" * 60)

    # Create datasets
    datasets = create_enhanced_datasets(english_sentences, french_sentences)
    en_train, fr_train = datasets['train']
    en_val, fr_val = datasets['val']
    en_test, fr_test = datasets['test']

    # Preprocess data (reuse your existing preprocessing)
    print("🔄 Preprocessing data...")
    train_x, train_y, x_tokenizer, y_tokenizer = preprocess(en_train, fr_train)
    val_x, val_y, _, _ = preprocess(en_val, fr_val)
    test_x, test_y, _, _ = preprocess(en_test, fr_test)

    # Get vocabulary sizes
    english_vocab_size = len(x_tokenizer.word_index) + 1
    french_vocab_size = len(y_tokenizer.word_index) + 1
    max_sequence_length = train_y.shape[1]

    print(f"📝 Vocabulary sizes: EN={english_vocab_size}, FR={french_vocab_size}")

    # Create enhanced model
    print("🏗️ Building Enhanced Transformer...")
    model = enhanced_transformer_model(
        train_x.shape,
        max_sequence_length,
        english_vocab_size,
        french_vocab_size
    )

    print(f"📋 Model Summary:")
    model.summary()

    # Prepare training data for the enhanced model
    train_encoder_input = train_x
    train_decoder_input = train_y[:, :-1]  # Exclude last token
    train_target = train_y[:, 1:]  # Exclude first token

    val_encoder_input = val_x
    val_decoder_input = val_y[:, :-1]
    val_target = val_y[:, 1:]

    # Enhanced training with callbacks
    callbacks = [
        tf.keras.callbacks.EarlyStopping(
            patience=3, restore_best_weights=True, monitor='val_loss'
        ),
        tf.keras.callbacks.ReduceLROnPlateau(
            factor=0.5, patience=2, min_lr=1e-6, monitor='val_loss'
        ),
        tf.keras.callbacks.ModelCheckpoint(
            'enhanced_model_best.h5', save_best_only=True, monitor='val_loss'
        )
    ]

    print("🎯 Training Enhanced Model...")
    start_time = time.time()

    history = model.fit(
        [train_encoder_input, train_decoder_input], train_target,
        batch_size=64,
        epochs=15,
        validation_data=([val_encoder_input, val_decoder_input], val_target),
        callbacks=callbacks,
        verbose=1
    )

    training_time = time.time() - start_time
    print(f"⏱️ Training completed in {training_time:.2f} seconds")

    # Enhanced evaluation
    print("📊 Evaluating Enhanced Model...")

    # Prepare test data
    test_data = list(zip(test_x, test_y))

    # Evaluate with beam search
    results = evaluate_enhanced_model(model, test_data[:100], y_tokenizer, beam_search=True)

    # Plot training history
    plt.figure(figsize=(12, 4))

    plt.subplot(1, 2, 1)
    plt.plot(history.history['loss'], label='Training Loss')
    plt.plot(history.history['val_loss'], label='Validation Loss')
    plt.title('Enhanced Model Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()

    plt.subplot(1, 2, 2)
    plt.plot(history.history['accuracy'], label='Training Accuracy')
    plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
    plt.title('Enhanced Model Accuracy')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.legend()

    plt.tight_layout()
    plt.show()

    # Show example translations
    print("\n🌟 Sample Enhanced Translations:")
    print("=" * 50)

    for i in range(min(5, len(en_test))):
        src_text = en_test[i]
        ref_text = fr_test[i]

        # Get model prediction
        src_tokens = x_tokenizer.texts_to_sequences([src_text])
        src_padded = pad_sequences(src_tokens, maxlen=train_x.shape[1], padding='post')

        if len(test_data) > i:
            pred_tokens = enhanced_beam_search_decode(model, src_padded[0], y_tokenizer, beam_size=4)
            pred_text = y_tokenizer.sequences_to_texts([pred_tokens])[0] if pred_tokens else "Unable to decode"
        else:
            pred_text = "Unable to generate"

        print(f"\n📝 Example {i+1}:")
        print(f"  🇬🇧 Source: {src_text}")
        print(f"  🇫🇷 Reference: {ref_text}")
        print(f"  🤖 Enhanced Prediction: {pred_text}")
        print(f"  📊 Simple BLEU: {results['bleu_scores'][i] if i < len(results['bleu_scores']) else 'N/A':.4f}")

    print("\n✅ Enhanced Training Pipeline Completed!")
    print("🎉 Key Improvements Applied:")
    print("  ✓ Modern Transformer architecture with pre-layer normalization")
    print("  ✓ Proper train/validation/test splits")
    print("  ✓ Label smoothing for better generalization")
    print("  ✓ Learning rate scheduling")
    print("  ✓ Beam search decoding")
    print("  ✓ Enhanced evaluation metrics")
    print("  ✓ Early stopping and model checkpointing")

    return {
        'model': model,
        'history': history,
        'results': results,
        'tokenizers': (x_tokenizer, y_tokenizer),
        'datasets': datasets
    }

# ============================================================================
# USAGE: Run this in your notebook after loading your data
# ============================================================================

print("🎯 Enhanced Machine Translation Ready!")
print("To use: run enhanced_training_pipeline(english_sentences, french_sentences)")
print()
print("Example:")
print("enhanced_results = enhanced_training_pipeline(english_sentences, french_sentences)")
print()
print("Features:")
print("  🔥 Modern Transformer with attention improvements")
print("  📊 Proper data splitting and evaluation")
print("  🎯 Beam search for better translations")
print("  📈 Advanced training techniques")
print("  🏆 Enhanced metrics and visualization")